# Mask R-CNN - Train on Llampuga Dataset

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import imgaug


# Root directory of the project
ROOT_DIR = os.path.abspath("")



# Import Mask RCNN
sys.path.append("")  # To find local version of the library
os.chdir("/mrcnn")
print(os.getcwd())
import inspect
import config

module = inspect.getmodule(config)
module_path = os.path.dirname(module.__file__)
print (module_path)

import utils
module1 = inspect.getmodule(config)
module_path1 = os.path.dirname(module1.__file__)
import model as modellib
import visualize
from model import log

  
    
os.chdir("")
%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
#COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

COCO_MODEL_PATH = os.path.join("/mask_rcnn_coco.h5")
#COCO_MODEL_PATH = os.path.join("/home/amaya/FOTOPEIX/MASK_RCNN/Mask_RCNN/logs/llampuga20200701T1205/mask_rcnn_llampuga_0029.h5")
print(COCO_MODEL_PATH)
# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
   # utils.download_trained_weights(COCO_MODEL_PATH)




## Configurations

In [ ]:
from datetime import *
from dataset import DatasetConfig
log_dir='/logs' 
config = DatasetConfig()
config.display()

date=datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
config.save(log_dir,date)


## Notebook Preferences

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Dataset

In [ ]:
from dataset import Dataset

In [ ]:
# Training dataset
dataset_train = Dataset()
dataset_train.load_dataset('', '/train3_merged_190_prueba.json')
dataset_train.prepare()

In [ ]:
# Validation dataset
dataset_val = Dataset()
dataset_val.load_dataset('', '/annotations/val2_merged.json')
dataset_val.prepare()

In [ ]:
dataset = dataset_train
image_ids =   np.random.choice(dataset.image_ids, 100) #np.array(dataset.image_ids)
print(image_ids)

for image_id in image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names)
    print (class_ids)

# TEST dataset

In [ ]:
dataset_test = Dataset()
dataset_test.load_dataset('/home/amaya/FOTOPEIX/MASK_RCNN/test_images/test_llampuga/test_llampuga_todas_cerca_fechas_training/', '/home/asabater/MASK_RCNN/data/toni_test/ann/coco_test.json')
dataset_test.prepare()

## Create Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
model.load_weights(COCO_MODEL_PATH, by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                   "mrcnn_bbox", "mrcnn_mask"])

## AUGMENTATION

In [ ]:
# The imgaug library is pretty flexible and make different types of augmentation possible. 
# The deterministic setting is used because any spatial changes to the image must also be 
# done to the mask. There are also some augmentors that are unsafe to apply. From the mrcnn
# library: 
# Augmentors that are safe to apply to masks: 
# ["Sequential", "SomeOf", "OneOf", "Sometimes","Fliplr", 
# "Flipud", "CropAndPad", "Affine", "PiecewiseAffine"]
# Affine, has settings that are unsafe, so always
# test your augmentation on masks

import imgaug as ia
from imgaug import augmenters as iaa

ia.seed(1)

# http://imgaug.readthedocs.io/en/latest/source/augmenters.html#sequential
seq_of_aug = iaa.Sequential([
    iaa.Crop(percent=(0, 0.1)), # random crops
    
    # horizontally flip 50% of the images
    iaa.Fliplr(0.5), 

    # Gaussian blur to 50% of the images
    # with random sigma between 0 and 0.5.
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    
    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    
    # Apply affine transformations to each image.
    # Scale/zoom them from 90% 5o 110%
    # Translate/move them, rotate them
    # Shear them slightly -2 to 2 degrees.
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-5, 5),
        shear=(-2, 2)
    )
], random_order=True) # apply augmenters in random order

In [ ]:
# Some example augmentations using the seq defined above.
image_id = np.random.choice(dataset_train.image_ids, 1)[0] image, image_meta, class_ids, bbox, mask = modellib.load_image_gt( dataset_train, config, image_id, use_mini_mask=False)
visualize.display_images( [image], titles=['original'])
image_list = [] for i in range(15): image_aug = seq_of_aug.augment_image(image) image_list.append( image_aug)
visualize.display_images(image_list, cols=5)

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train bcy name pattern.
import keras
model.keras_model.metrics_tensors = []
augmentation=seq_of_aug
augmentation = imgaug.augmenters.Sometimes(0.5, [
                    imgaug.augmenters.Fliplr(0.5),
                    #imgaug.augmenters.Flipud(0.5),
                    #imgaug.augmenters.Affine(
                                   # rotate=(-15,15),  # rotate by -45 to +45 degrees
                                   # shear=(-2, 2),  # shear by -16 to +16 degrees
                                    #order=1,  # order=[0, 1],  # use nearest neighbour or bilinear interpolation (fast)
                                   # cval=0,  # cval=(0, 255),  # if mode is constant, use a cval between 0 and 255
                                  # mode='constant'
                                    ## mode=ia.ALL  # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                              # )
                ])

#augmentation1 = imgaug.augmenters.Sometimes(5/6,imgaug.augmenters.OneOf(
                                            #[
                                           # imgaug.augmenters.Fliplr(0.5), 
                                           # imgaug.augmenters.Flipud(0.5), 
                                            #imgaug.augmenters.Affine(rotate=(90)) 
                                           # imgaug.augmenters.Affine(rotate=(-15, 15)) 
                                           # ]                                             
                                       # )
                                  # )
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=100, 
            layers='all',
            augmentation=seq_of_aug
           )
#model.train(dataset_train, dataset_val, 
           #learning_rate=config.LEARNING_RATE ,
            # epochs=50, 
           #  layers="all",
            # augmentation=augmentation
           # )
 


## Evaluation

In [ ]:
from dataset import InferenceConfig

inference_config = InferenceConfig()

#bad_testing_samples = [9]

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
#model_path = os.path.join(ROOT_DIR, "../data/models/mask_rcnn_hake_0100.h5")
#model_path = os.path.join("/home/instman/FOTOPEIX/INSTANCE_SEGMENTATION/2018-dlcv-team5-master/Mask_RCNN/data/models/mask_rcnn_hake_0100.h5")
model_path = os.path.join("/llampuga20200716T1119/mask_rcnn_llampuga_0068.h5")
# model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

## EVALUACIÓN DE RESULTADOS

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 5 images. Increase for better accuracy.

image_ids = np.random.choice(dataset_val.image_ids,20)  #np.array(dataset_val.image_ids) #

APs = []
prs = []
rca = []
ovl = []
#a=image_ids[7]
savedir=''
#bad_testing_samples = [8,24]

for image_id in image_ids:
    mat=[]
    mat1=[]
    # Skip wrongly tagged images (ugly, but we could not understand why tags for these images is not working)
    #if image_id in bad_testing_samples:
        # continue
    # Load image and ground truth data
    print('image id: ' + str(image_id))
    
    
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    #if   r['masks'].all()== False:
        #break
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    results = model.detect([image], verbose=0)
    r = results[0]
   

    #visualize.display_instances(image_id, gt_bbox, gt_mask, gt_class_id, 
                            #dataset_val.class_names, ax=get_ax(size=20))

#visualize.display_weight_stats(model)
    
    filename=str(image_id)
    visualize.display_differences(image,
                        gt_bbox, gt_class_id, gt_mask,
                        r['rois'], r['class_ids'], r['scores'],r['masks'],
                        dataset_val.class_names, title="", ax=get_ax(size=20),
                        show_mask=True, show_box=True,
                        iou_threshold=0.5, score_threshold=0.5)
    
    plt.savefig(savedir+filename+'_differences.jpg')
    
    
    ma=len(results[0]['rois'])
    mi=list(map(str, range(ma)))

    visualize.display_instances(image, r['rois'],  r['masks'],r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax(size=20),captions=mi)
    plt.savefig(savedir+filename+'_segmentation.jpg')
    
   #print( dataset_test.source_image_link)  
    
            
   # np.savetxt(savedir+filename+'_score.txt', r['scores'], fmt='%1.3f')
    
   # np.savetxt(savedir+filename+'_class_ids.txt', r['class_ids'], fmt='%4d')
   # np.savetxt(savedir+filename+'_class_ids_gt.txt', gt_class_id, fmt='%4d')
    #np.savetxt(savedir+filename+'_bbox_gt.txt', gt_bbox, fmt='%4d')                  
    
    rois = results[0]['rois']
    for i, r in enumerate(rois):
        y1, x1, y2, x2 = r
        
        m=[x1,x2,y1,y2]
        
        mat.append(m)    
   # np.savetxt(savedir+filename+'_bbox.txt', mat,fmt='%4d %4d %4d %4d')   
   
    #print("precision: ", prs)
    #print("recall: ", rca)
    #print("overlaps: ", ovl)


   # np.savetxt(savedir+filename+'pre_reca.txt', np.column_stack([precisions,recalls]))

print("mAP: ", np.mean(APs))

#np.savetxt(savedir+'class_names.txt', dataset_val.class_names, fmt='%s')  

## MATRIZ DE CONFUSION

In [ ]:
import pandas as pd
gt_tot = np.array([])
pred_tot = np.array([])
mAP_ = []

image_ids = np.random.choice(dataset_val.image_ids,1)


for image_id in image_ids:

    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, config, image_id, use_mini_mask=False)
    info = dataset_val.image_info[image_id]
    #print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id,
    #dataset.image_reference(image_id)))
    # Run object detection
    results = model.detect([image], verbose=1)

    # Display results
    #ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax(size=20),
                            title="Predictions")
    #log("gt_class_id", gt_class_id)
    #log("gt_bbox", gt_bbox)
    #log("gt_mask", gt_mask)
    gt, pred = utils.gt_pred_lists(gt_class_id, gt_bbox, r['class_ids'], r['rois'], iou_tresh = 0.4)
    gt_tot = np.append(gt_tot, gt)
    pred_tot = np.append(pred_tot, pred)
    #precision_, recall_, AP_ = utils.compute_precision_recall_map(gt_tot, pred_tot)
    AP_, precision_, recall_, overlap_ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                      r['rois'], r['class_ids'], r['scores'], r['masks'], iou_threshold = 0.4)
    print("the actual len of the gt vect is : ", len(gt_tot))
    print("the actual len of the pred vect is : ", len(pred_tot))
    print("the actual precision is : ", precision_)
    print("the actual recall is : ", recall_)
#print("the actual overlaps is : ", overlap_)
    mAP_.append(AP_)
    print("the actual average precision : ", AP_)
    print("the actual mean average precision : ", sum(mAP_)/len(mAP_))

directory = 'output'
gt_pred_tot_json = {"gt_tot" : gt_tot.astype(int), "pred_tot" : pred_tot.astype(int)}
df = pd.DataFrame(gt_pred_tot_json)

import os
if not os.path.exists(directory):
    os.makedirs(directory)
df.to_json(os.path.join(directory, 'gt_pred_test.json'))

from mrcnn import confusion_matrix_pretty_print

confusion_matrix_pretty_print.plot_confusion_matrix_from_data( gt_tot, pred_tot,columns=None, annot=True, cmap="Oranges",
      fmt='.2f', fz=11, lw=0.5, cbar=False, figsize=[8,8], show_null_values=0, pred_val_axis='lin')

## INFERENCIA  EN IMÁGENES DE LLAMPUGA DE LONJA

In [ ]:
import os
import numpy as num
from PIL import Image
from PIL.ExifTags import TAGS
name=[]
data1=[]
vta=[]
ord1=[]
emb=[]
pes=[]
caj=[]
fao=[]
DATO=[]
savedir=''

#DEBE ENTRAR EN LAS CARPETAS QUE SE ENCUENTRE CON EL NOMBRE OPPM_Subasta_"fecha"

for root, dirs, files in os.walk("/OPMM_Subasta_2020-09-01/"):  
    for filename in files:
        mat=[]

        image=Image.open(root+filename)
        exifdata = image._getexif()
        
        for tag_id in exifdata:
#    # get the tag name, instead of human unreadable tag id
            tag = TAGS.get(tag_id, tag_id)
            data = exifdata.get(tag_id)
#    # decode bytes 
        if isinstance(data, bytes):
            
            data = data.decode()
               
            fao= data[96] + data[98] + data[100]
            ord1 = data[28]
            caj= data[82] + data[84] 
               
            if (fao == 'DOL') and (ord1 == '1') and (caj == '01'):
                print(filename)
                vta=data[10] +data[12]+data[14]+data[16]   
                emb=data[40]+data[42]+data[44]+data[47]
                pes= data[60]+data[62]+data[64]+data[66]+data[68]
                #DAT =  num.column_stack((filename,vta,ord1,emb,pes,caj, fao))
               # DATO.append(DAT)
                    #####EJECUTAR INFERENCIA DE MASKRCNN 
                image=cv2.imread(root+filename)
                print(filename)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image, window, scale, padding, crop = utils.resize_image(
                image,
                min_dim=config.IMAGE_MIN_DIM,
                min_scale=config.IMAGE_MIN_SCALE,
                max_dim=config.IMAGE_MAX_DIM,
                mode=config.IMAGE_RESIZE_MODE)
       
        
                results = model.detect([image], verbose=1)

                r = results[0]
      
                ma=len(results[0]['rois'])
                mi=list(map(str, range(ma)))
                rois = results[0]['rois']
                
                
                visualize.display_instances(image, r['rois'],  r['masks'],r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax(size=20),captions=mi)
         
                np.savetxt(savedir+filename+'_score.txt', r['scores'], fmt='%1.3f')
    
                np.savetxt(savedir+filename+'_class_ids.txt', r['class_ids'], fmt='%4d')
                for i, r in enumerate(rois):
                    y1, x1, y2, x2 = r
        
                    m=[x1,x2,y1,y2]
        
                    mat.append(m)
                np.savetxt(savedir+filename+'_bbox.txt', mat,fmt='%4d %4d %4d %4d')
                #np.savetxt(savedir+'DATO.txt',DATO,fmt='%s %s %s %s %s %s %s')
            else:
                     
                print(filename + ' no llampuga')